In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook performs maximum likelihood estimation (MLE) for one participant's choices, under a feature reinforcement learning with decay model. The fitting procedure tests best fit (MLE) parameters on choice data from a left-out game.

In [24]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet
from scipy.optimize import minimize
import time as time
import pickle 

# Custom dependencies
import os
from World import World
from FeatureRL import Agent as FRL
from FeatureRL import train_frl_choice
from Data import Data, extract_vars 
from fitting import Fit 

## 1. Instantiate world

In [25]:
## Set world properties.
n_dims = 3
n_feats_per_dim = 3
world_h = 0
world_p_high = 0.75
world_p_low = 0.25
outcome = 1
 
world = World(n_dims, n_feats_per_dim, world_h, world_p_high, world_p_low, outcome)

## 2. Load data

In [26]:
## Set paths.
behav_path = os.getcwd() + '/subj1.csv'
fit_path = os.getcwd() + '/subj1_MLE_fit.obj'
fit_file = open(fit_path, 'wb')

## Load csv. 
behav_data = pd.read_csv(behav_path)

## Make data object.
data = Data(behav_data)

## 3. Run MLE with leave-one-game out cross-validation 

In [29]:
start_time = time.time()

## Set number of iterations.
n_iter = 5

## Set bounds.
bnds = [(1e-6,1), (1e-6,1), (1e-6,500)]

## Initialize output arrays.
results = np.zeros((n_iter,data.n_games,len(bnds)+1))
initial_conditions = np.zeros((n_iter,len(bnds)))

## Run optimization with different initial conditions.
for i in np.arange(n_iter): 
    
    print('iteration #', i+1)
    
    ## Set initial conditions.
    init_cond = [np.random.uniform(0, 1), np.random.uniform(0, 1), np.random.uniform(2, 5)]
    initial_conditions[i,:] = init_cond
    print('initial conditions:', init_cond)
        
    for g in np.arange(data.n_games):
    
        ## Set current test game. 
        test_game = g+1
        print('predicting game:', test_game)

        ## Split behavior into training and test data.
        behav_training_data, behav_test_data = data.split_data(test_game)

        ## Train model.
        res = minimize(train_frl_choice, init_cond, args=(behav_training_data), method='L-BFGS-B', bounds=bnds, options={'maxfun': 150, 'maxiter': 100})
        results[i,test_game-1,0:len(bnds)] = res.x

        ## Evaluate model on test set.
        test_trials = behav_test_data.Trial.unique()        
        extracted_data = extract_vars(behav_test_data, test_trials)

        ## Set parameters.
        # Default values set to 0.
        test_params = {'learning_rate': res.x[0],
                    'decay_rate': res.x[1],
                    'softmax_temperature': res.x[2],
                    'w_init': 0,
                    'decay_target': 0,
                    'precision': 0}
        del res

        ## Instantiate and run MLE agent.
        frl_mle = FRL(world, test_params)
        W, test_lik = frl_mle.choice_likelihood(world, extracted_data)
        results[i,test_game-1,-1] = test_lik

        n_trials = np.shape(extracted_data['outcomes'])
        
        print('maximum likelihood learning rate:', test_params['learning_rate'])
        print('maximum likelihood decay rate:', test_params['decay_rate'])
        print('maximum likelihood beta on value:', test_params['softmax_temperature'])
        print('test set log likelihood:', test_lik)
        
        print('test set likelihood per trial:', np.exp(test_lik/n_trials))
        
elapsed_time = time.time() - start_time
print('elapsed time: ' + str((np.round(elapsed_time,decimals=3))) + ' seconds')

iteration # 1
initial conditions: [0.4135765898362317, 0.5567194749751273, 3.908749346233261]
predicting game: 1
total training set log likelihood: -213.978354575391
total training set log likelihood: -213.97835665986793
total training set log likelihood: -213.97835438795414
total training set log likelihood: -213.97835455128836
total training set log likelihood: -347.1577676865943
total training set log likelihood: -347.15773053174274
total training set log likelihood: -347.1577676865869
total training set log likelihood: -347.1577676865885
total training set log likelihood: -193.98246551583227
total training set log likelihood: -193.98246533689903
total training set log likelihood: -193.98246572480394
total training set log likelihood: -193.98246545112974
total training set log likelihood: -193.45415324888842
total training set log likelihood: -193.4541531005889
total training set log likelihood: -193.4541534596583
total training set log likelihood: -193.4541531877297
total training 

total training set log likelihood: -188.1545154311152
total training set log likelihood: -188.1545159101216
total training set log likelihood: -188.1545154211805
total training set log likelihood: -188.15451543101784
total training set log likelihood: -187.7123459053363
total training set log likelihood: -187.7123456777581
total training set log likelihood: -187.71234601799495
total training set log likelihood: -187.7123459058658
total training set log likelihood: -187.21976540231972
total training set log likelihood: -187.21976544120452
total training set log likelihood: -187.21976542100435
total training set log likelihood: -187.219765404478
total training set log likelihood: -187.20131119105667
total training set log likelihood: -187.20131123393224
total training set log likelihood: -187.20131119085121
total training set log likelihood: -187.2013111928185
total training set log likelihood: -187.1945882360195
total training set log likelihood: -187.1945882480051
total training set lo

total training set log likelihood: -192.35728725816037
total training set log likelihood: -192.35728721813808
total training set log likelihood: -192.3572871768696
total training set log likelihood: -192.1552636054931
total training set log likelihood: -192.15526360226357
total training set log likelihood: -192.15526362447972
total training set log likelihood: -192.15526360294237
total training set log likelihood: -192.1129155490034
total training set log likelihood: -192.1129155392975
total training set log likelihood: -192.11291554914806
total training set log likelihood: -192.11291554850987
total training set log likelihood: -192.11229986024165
total training set log likelihood: -192.11229985919417
total training set log likelihood: -192.11229986005605
total training set log likelihood: -192.11229986018543
total training set log likelihood: -192.11229027945532
total training set log likelihood: -192.11229027948994
total training set log likelihood: -192.1122902794401
total training 

KeyboardInterrupt: 

In [ ]:
## Initialize and save fit object.
fit = Fit(subj_id, n_iter, initial_conditions, results, elapsed_time)
pickle.dump(fit, fit_file)
fit_file.close()

In [ ]:
## Test that fit was saved correctly. 
new_fit_file = open(fit_path,'rb')
loaded_fit = pickle.load(new_fit_file)

In [ ]:
loaded_fit.elapsed_time